In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pathlib


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, LeakyReLU, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
images = pathlib.Path('dataset_normal')

In [4]:
train_data = image_dataset_from_directory(images,
                                          validation_split=0.2,
                                          subset='training',
                                         seed=123,
                                         image_size=(360,360))

Found 17092 files belonging to 8 classes.
Using 13674 files for training.


In [5]:
val_data= image_dataset_from_directory(images,
                                          validation_split=0.2,
                                          subset='validation',
                                         seed=123,
                                         image_size=(360,360))

Found 17092 files belonging to 8 classes.
Using 3418 files for validation.


In [6]:
for image, labels in train_data:
    print(image.shape)
    print(labels.shape)
    break
    
print(train_data.class_names)

(32, 360, 360, 3)
(32,)
['basophil', 'eosinophil', 'erythroblast', 'ig', 'lymphocyte', 'monocyte', 'neutrophil', 'platelet']


## Modèle freeze

In [8]:
n_class = 8
base_model = InceptionV3(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False    

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=n_class+1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

87924736/87910968 [==============================] - 3s 0us/step


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 4

In [10]:
history = model.fit(train_data, 
                    validation_data=val_data, 
                    epochs=5)

Epoch 1/5
428/428 [==============================] - 577s 1s/step - loss: 3.3764 - acc: 0.3900 - val_loss: 1.2055 - val_acc: 0.5155
Epoch 2/5
428/428 [==============================] - 571s 1s/step - loss: 1.3785 - acc: 0.4505 - val_loss: 1.2133 - val_acc: 0.5512
Epoch 3/5
428/428 [==============================] - 573s 1s/step - loss: 1.3917 - acc: 0.4481 - val_loss: 1.2594 - val_acc: 0.5222
Epoch 4/5
428/428 [==============================] - 572s 1s/step - loss: 1.3689 - acc: 0.4751 - val_loss: 1.2429 - val_acc: 0.5559
Epoch 5/5
428/428 [==============================] - 571s 1s/step - loss: 1.3676 - acc: 0.4843 - val_loss: 1.2049 - val_acc: 0.5846


In [11]:
images = pathlib.Path('TestA')
test_data = image_dataset_from_directory(images,
                                         seed=123,
                                        subset=None,
                                         image_size=(360,360))

Found 4350 files belonging to 8 classes.


In [12]:
model.evaluate(test_data)

136/136 [==============================] - 145s 1s/step - loss: 2.9367 - acc: 0.0522


[2.936724901199341, 0.052183907479047775]

## Modèle unfreeze sur 4 couches

In [13]:
for layer in base_model.layers[-4:]: #defreeze 4 couches
    layer.trainable = True

In [ ]:
history_defreeze4 = model.fit(train_data, 
                    validation_data=val_data, 
                    epochs=10)

Epoch 1/10
428/428 [==============================] - 573s 1s/step - loss: 1.3437 - acc: 0.4981 - val_loss: 1.1408 - val_acc: 0.6164
Epoch 2/10
428/428 [==============================] - 575s 1s/step - loss: 1.3116 - acc: 0.5205 - val_loss: 1.1042 - val_acc: 0.6118
Epoch 3/10
428/428 [==============================] - 574s 1s/step - loss: 1.2926 - acc: 0.5303 - val_loss: 1.1204 - val_acc: 0.6047
Epoch 4/10
147/428 [=========>....................] - ETA: 5:01 - loss: 1.3090 - acc: 0.5172

In [ ]:
model.evaluate(test_data)